In [1]:
import os
from dotenv import load_dotenv
import praw
from datetime import datetime
import pandas as pd
from typing import List, Dict, Any
import requests
import gradio as gr
import asyncio

# Charger les variables d'environnement
load_dotenv(override=True)

# Configuration Reddit
REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT = "RedditScraper/1.0"

print("🔧 Configuration Reddit:")
print(f"Client ID: {REDDIT_CLIENT_ID[:10]}..." if REDDIT_CLIENT_ID else "❌ Non défini")
print(f"Client Secret: {REDDIT_CLIENT_SECRET[:10]}..." if REDDIT_CLIENT_SECRET else "❌ Non défini")

🔧 Configuration Reddit:
Client ID: KrR4WKqaRf...
Client Secret: wnsAK-PrFa...


In [2]:
from openai import OpenAI
from agents import Agent, Runner, function_tool, trace, WebSearchTool
# Créer le client OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

agent = Agent(name="Assistant", instructions="You are a helpful assistant")


In [3]:
agent = Agent(name="Assistant", instructions="Tu es un manager de projet intelligent, structuré, qui comprend les besoins et les dépendances entre agents.\
    Tu comprends les besoins d'un projet et propose une bonne ou meilleure structuration",
           model="gpt-4o",
)

In [5]:
message = """OK, maintenant je te donne l'architecture complète de ce que je veux faire. Vois des choses à améliorer?

des pompts pour les agents ? Une meilleur structuration ? Tes instructions seront ensuite données à Claude
afin qu'il code ce que tu dis. 

<début>

Le projet doit être capable de gérer les demandes dans toutes les langues.

Il y aura 4 agents.

La structure de Reddit est ainsi : Reddit → Subreddit → Post → Commentaire.

Le premier sera celui en contact direct avec l'utilisateur. Il devra être poli, dire ce qu’il fait, c’est-à-dire chercher les points de douleur/friction sur les subreddits afin de trouver des solutions/idées, et toujours rediriger vers son objectif, à savoir : quel subreddit scraper, combien de posts, combien de commentaires par post, ainsi que quels critères parmi top, new, best, rising, hot. Si l'utilisateur ne sait pas, il doit lui expliquer ce que sont ces critères et comment ils sont calculés.

Il aura deux outils : un pour la recherche internet au cas où, et un autre pour vérifier si le subreddit existe.
Si oui, c’est bon. Sinon, il redirigera soit :

en cas de faute d’orthographe, vers le subreddit correctement orthographié ;

sinon, vers un subreddit proche de ce que l'utilisateur voulait ;

et par défaut, vers les subreddits les plus courants pour les entrepreneurs.

Il doit prévenir l'utilisateur et lui indiquer ses paramètres par défaut, qui sont : 10 posts, 10 commentaires, et le critère top. Avant de passer à l'étape suivante, il doit s’assurer que l'utilisateur n’a rien à ajouter, en posant une dernière question.

L'étape suivante est un handoff : il transmet les critères et le subreddit.
Le second agent est un scraper. Il scrape tout selon les critères, en utilisant un outil dédié. Une fois qu’il a terminé, il effectue un handoff vers un troisième agent.

Ce troisième agent est spécialisé dans les résumés, l’analyse de sentiment et le scoring.

Tu es un expert analyste spécialisé dans l’identification des problèmes et frustrations récurrents des utilisateurs.

Ton rôle est de :

Analyser le sentiment et l’intensité émotionnelle de chaque post/commentaire ;

Identifier les douleurs récurrentes, quelle que soit la langue ;

Calculer un score de priorité pour chaque douleur ;

Classer les douleurs par ordre de priorité ;

Lister les 5 principales douleurs sous forme d’une phrase chacune dans ton résumé final.

Instructions spécifiques :

Analyse le sentiment principal (ex. : frustration, colère, stress, confusion, etc.) — une émotion principale par commentaire ;

Évalue l’intensité émotionnelle sur une échelle de 1 à 10 (10 = très intense) ;

Identifie le type de problème (technique, financier, relationnel, etc.) ;

Regroupe les douleurs similaires ;

Calcule les métriques associées à chaque douleur ;

Présente les 3 douleurs principales avec leur score.

Format de sortie attendu :

Liste jusqu’à 10 principales douleurs/problèmes rencontrés (une phrase chacune) ;

Top 3 douleurs classées par score ;

Résumé en un paragraphe ;

Si des commentaires sont particulièrement intéressants car ils proposent explicitement une solution, garde-les en mémoire et demande à l'utilisateur s’il souhaite les lire ou obtenir les liens.

Enfin, il devra faire un handoff vers un dernier agent spécialisé dans les recommandations. Il donnera au quatrième agent son résumé ainsi que les commentaires qu’il a mis en mémoire.

Le quatrième agent est créatif, intelligent, et cherche à résoudre les problèmes de la manière la plus simple et la plus efficace possible. Il proposera, pour chaque problème rencontré dans le rapport précédent, 3 projets/idées pour les résoudre. 
Important: les idées/résolutions de problèmes doivent être le plus concret et simple possible
avec des idées de SAAS et produits numériques en priorités.
Enfin, il retournera le rapport du troisième agent ainsi que ses recommandations.

Comme outils, il proposera une exportation en PDF, CSV ou TXT de son rapport, ainsi que la possibilité :

soit de lister les commentaires marquants qu’il a en mémoire avec leur référence ;

soit de les exporter également en PDF, CSV ou TXT.
</fin>
"""

In [7]:
history = """Pour une structuration optimale, voici quelques suggestions d'amélioration :

### Clarification des rôles et interactions

1. **Agent 1 : Interaction Utilisateur**
   - **Tâches** : Politesse, guider l'utilisateur, valider les critères de scraping.
   - **Améliorations** : 
     - Ajoutez une étape de confirmation finale avec l'utilisateur pour consolider les paramètres avant le handoff.
     - Incluez des exemples concrets pour chaque critère (top, new, etc.) pour faciliter la compréhension.

2. **Agent 2 : Scraping des Données**
   - **Tâches** : Scraping selon les critères établis.
   - **Améliorations** : 
     - Assurez la compatibilité multilingue du scraper pour garantir une collecte de données sans faille.

3. **Agent 3 : Analyse et Synthèse**
   - **Tâches** : Sentiment analysis, identification des problèmes.
   - **Améliorations** : 
     - Implémentez un système de filtre pour éliminer les faux positifs dans l'analyse sentimentale.
     - Précisez comment les scores de priorité sont calculés.

4. **Agent 4 : Recommandations**
   - **Tâches** : Proposer des solutions concrètes.
   - **Améliorations** : 
     - Validez la faisabilité des suggestions avec des exemples ou études de cas.
     - Encouragez l'utilisation de frameworks existants pour les idées de SAAS.

### Optimisation des Outils

1. **Agent 1 :**
   - Intégrez des outils de vérification linguistique pour une gestion efficace des fautes d’orthographe.

2. **Agent 2 :**
   - Choisissez des outils de scraping robustes assurant la conformité avec les normes d’utilisation des données de Reddit.

3. **Agent 3 :**
   - Utilisez des bibliothèques éprouvées pour l'analyse de sentiment multilingue.

4. **Agent 4 :**
   - Optez pour des bibliothèques d’IA génératives pour formuler des solutions innovantes et pratiques.

### Coordination et Gestion de Projet

- Assurez une communication fluide entre les agents en utilisant des API standardisées et un format de données commun (JSON, XML).
- Définissez des indicateurs de performance clé (KPI) pour évaluer l'efficacité de chaque agent.
- Prévoyez un mécanisme de feedback pour améliorer continuellement le système en fonction des interactions utilisateur.

En intégrant ces améliorations, vous renforcerez la robustesse et l'efficacité de votre projet tout en optimisant l'expérience utilisateur.
"""

In [10]:
question = "Tu m'as donné les améliorations. Maintenant fait un TOUT COMPLET pour que je puisse le donner à claude \
    Il doit tout y avoir, les prompts des agents, les outils tout !!"

In [11]:
with trace("Structure"):
    result = await Runner.run(agent,question + history + message)
    print(result.final_output)

Pour structurer ce projet de manière optimale, voici mon retour :

### Structure Optimisée des Agents et Interactions

#### Agent 1 : Interaction Utilisateur
- **Tâches** :
  - Saluer l'utilisateur et expliquer le processus.
  - Clarifier les critères (top, new, best, rising, hot) et l'architecture Reddit.
  - Confirmer les paramètres : subreddit, nombre de posts/commentaires et critères.
- **Outils** :
  - Recherche Internet pour vérifier et expliquer les subreddits.
  - Vérificateur d'existence de subreddit avec correction automatique.
- **Prompts** :
  - "Bonjour ! Je vais vous aider à identifier des points de douleur sur Reddit. Pouvez-vous me dire quel subreddit vous intéresse et vos préférences de recherche ?"
  - "Voici nos paramètres par défaut : 10 posts, 10 commentaires, critère top. Avez-vous des changements à apporter ?"
- **Améliorations** :
  - Prévoir des exemples concrets lors de l’explication des critères.

#### Agent 2 : Scraping des Données
- **Tâches** :
  - Scraper

In [ ]:
with trace("test"):
    result = await Runner.run(agent, "Write a haiku about recursion in programming")
    print(result.final_output)

In [ ]:
system_prompt = """Tu es un assistant spécialisé pour aider les entrepreneurs à trouver des problèmes à résoudre sur
 Reddit.

Ton rôle est de:
1. Demander le subreddit à analyser
2. Proposer les options de tri disponibles:
   - �� "Top" → posts avec le meilleur score (upvotes - downvotes) sur une période
   - 🆕 "New" → les plus récents
   - 💬 "Hot" → combinaison de score + fraîcheur (post récent avec beaucoup d'activité)
   - 🌟 "Best" → pertinence + vote + réponse (dans les commentaires)
3. Demander combien de posts analyser
4. Utiliser tes outils pour vérifier l'existence du subreddit et rechercher des informations si nécessaire

Tu peux utiliser comme outils:
- check_subreddit_exists: pour vérifier si un subreddit existe
- WebSearchTool: pour rechercher des informations récentes sur les subreddits ou problèmes entrepreneurs

Sois toujours poli, professionnel et aide l'utilisateur à faire le meilleur choix pour ses besoins."""

In [ ]:
# Outil pour vérifier si un subreddit existe
@function_tool
def check_subreddit_exists(subreddit_name: str) -> Dict:
    """
    Vérifie si un subreddit existe en faisant une requête à Reddit
    """
    try:
        url = f"https://www.reddit.com/r/{subreddit_name}/about.json"
        headers = {
            'User-Agent': 'RedditScraper/1.0'
        }
        
        response = requests.get(url, headers=headers, timeout=10)
        print(response.status_code)
        
        if response.status_code == 200:
            data = response.json()
            if 'data' in data and 'subscribers' in data['data']:
                return {
                    "exists": True,
                    "subscribers": data['data']['subscribers'],
                    "description": data['data'].get('public_description', ''),
                    "title": data['data'].get('title', '')
                }
        
        return {"exists": False, "error": "Subreddit non trouvé"}
        
    except Exception as e:
        return {"exists": False, "error": str(e)}


In [ ]:
# Test de l'outil
print("Test de vérification de subreddit:")
test_subreddits = ["france", "programming", "nonexistentsubreddit123"]
for sub in test_subreddits:
    result = check_subreddit_exists(sub)
    print(f"r/{sub}: {'✅ Existe' if result['exists'] else '❌ N\'existe pas'}")

In [ ]:
agent = Agent(
    name="Assistant",
    tools=[
        WebSearchTool(),
        check_subreddit_exists,
    ],
)

In [ ]:
message = "prix actions tesla aujourdhui, utilise recherche web"

with trace("test"):
    result = await Runner.run(agent, message)
    print(result.final_output)

In [ ]:
def chat(message, history):

    response = Runner.run(agent, message)
    reply =response.choices[0].message.content      
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

In [ ]:
async def chat(message, history):
    """
    Fonction de chat asynchrone pour Gradio
    """
    try:
        with trace("chat_interaction"):
            result = await Runner.run(agent, message)
            return result.final_output
    except Exception as e:
        return f"Erreur: {str(e)}"

# Interface Gradio avec fonction asynchrone
gr.ChatInterface(chat, type="messages").launch()

In [ ]:
async def chat_with_history(message, history):
    """
    Fonction de chat avec historique intégré de Gradio
    """
    try:
        # Construire le contexte avec l'historique
        full_context = ""
        for human, assistant in history:
            full_context += f"Humain: {human}\nAssistant: {assistant}\n"
        
        # Ajouter le message actuel
        full_context += f"Humain: {message}\nAssistant: "
        
        with trace("chat_with_history"):
            result = await Runner.run(agent, full_context)
            return result.final_output
            
    except Exception as e:
        return f"Erreur: {str(e)}"

# Interface avec historique automatique
gr.ChatInterface(
    chat_with_history, 
    title="Assistant Reddit",
    description="Posez vos questions sur Reddit et les entrepreneurs",
    examples=[
        "Vérifie si le subreddit 'entrepreneur' existe",
        "Quels sont les problèmes courants des entrepreneurs ?",
        "Recherche des informations sur les startups en France"
    ]
).launch()

In [ ]:
# Configuration Reddit (utilise les variables déjà définies)
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

@function_tool
def scrape_subreddit_top_posts(subreddit_name: str, num_posts: int = 10) -> Dict[str, Any]:
    """
    Scrape les posts avec le meilleur score d'un subreddit donné
    
    Args:
        subreddit_name: Nom du subreddit (sans le 'r/')
        num_posts: Nombre de posts à récupérer (défaut: 10)
    
    Returns:
        Dict contenant les posts scrapés avec leurs détails
    """
    try:
        # Vérifier que le subreddit existe
        subreddit = reddit.subreddit(subreddit_name)
        
        # Récupérer les posts avec le meilleur score
        posts_data = []
        
        for post in subreddit.top(limit=num_posts, time_filter='month'):
            # Récupérer les commentaires principaux
            post.comments.replace_more(limit=5)  # Limiter les commentaires imbriqués
            
            comments_data = []
            for comment in post.comments.list()[:10]:  # Top 10 commentaires
                if hasattr(comment, 'body') and comment.body:
                    comments_data.append({
                        "author": str(comment.author) if comment.author else "[deleted]",
                        "body": comment.body,
                        "score": comment.score,
                        "created_utc": datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                    })
            
            post_data = {
                "title": post.title,
                "author": str(post.author) if post.author else "[deleted]",
                "score": post.score,
                "upvote_ratio": post.upvote_ratio,
                "num_comments": post.num_comments,
                "created_utc": datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                "url": f"https://reddit.com{post.permalink}",
                "selftext": post.selftext[:500] + "..." if len(post.selftext) > 500 else post.selftext,
                "comments": comments_data
            }
            
            posts_data.append(post_data)
        
        return {
            "success": True,
            "subreddit": subreddit_name,
            "posts_count": len(posts_data),
            "posts": posts_data,
            "scraped_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "subreddit": subreddit_name
        }

# Test de la fonction
print("🧪 Test du scraping Reddit:")
test_result = scrape_subreddit_top_posts("programming", 3)
print(f"✅ Posts récupérés: {test_result['posts_count'] if test_result['success'] else '❌ Erreur'}")
if test_result['success']:
    for i, post in enumerate(test_result['posts'], 1):
        print(f"  {i}. {post['title'][:50]}... (Score: {post['score']})")

In [ ]:
@function_tool
def scrape_subreddit_posts(subreddit_name: str, num_posts: int = 10, sort_criteria: str = "top") -> Dict[str, Any]:
    """
    Scrape les posts d'un subreddit selon différents critères de tri
    
    Args:
        subreddit_name: Nom du subreddit (sans le 'r/')
        num_posts: Nombre de posts à récupérer (défaut: 10)
        sort_criteria: Critère de tri - "top", "new", "hot", "best", "rising" (défaut: "top")
    
    Returns:
        Dict contenant les posts scrapés avec leurs détails
    """
    try:
        # Vérifier que le subreddit existe
        subreddit = reddit.subreddit(subreddit_name)
        
        # Mapper les critères de tri aux méthodes PRAW
        sort_methods = {
            "top": subreddit.top,
            "new": subreddit.new,
            "hot": subreddit.hot,
            "best": subreddit.best,
            "rising": subreddit.rising
        }
        
        # Vérifier que le critère est valide
        if sort_criteria not in sort_methods:
            sort_criteria = "top"  # Défaut si critère invalide
        
        # Récupérer les posts selon le critère choisi
        posts_data = []
        
        # Appliquer le tri avec time_filter pour "top" et "rising"
        if sort_criteria in ["top", "rising"]:
            posts = sort_methods[sort_criteria](limit=num_posts, time_filter='month')
        else:
            posts = sort_methods[sort_criteria](limit=num_posts)
        
        for post in posts:
            # Récupérer les commentaires principaux
            post.comments.replace_more(limit=5)
            
            comments_data = []
            for comment in post.comments.list()[:10]:
                if hasattr(comment, 'body') and comment.body:
                    comments_data.append({
                        "author": str(comment.author) if comment.author else "[deleted]",
                        "body": comment.body,
                        "score": comment.score,
                        "created_utc": datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                    })
            
            post_data = {
                "title": post.title,
                "author": str(post.author) if post.author else "[deleted]",
                "score": post.score,
                "upvote_ratio": post.upvote_ratio,
                "num_comments": post.num_comments,
                "created_utc": datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                "url": f"https://reddit.com{post.permalink}",
                "selftext": post.selftext[:500] + "..." if len(post.selftext) > 500 else post.selftext,
                "comments": comments_data
            }
            
            posts_data.append(post_data)
        
        return {
            "success": True,
            "subreddit": subreddit_name,
            "sort_criteria": sort_criteria,
            "posts_count": len(posts_data),
            "posts": posts_data,
            "scraped_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "subreddit": subreddit_name,
            "sort_criteria": sort_criteria
        }

In [ ]:
import json
import csv
from pathlib import Path
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

@function_tool
def export_analysis_to_txt(analysis_text: str, filename: str = "reddit_analysis.txt") -> Dict[str, str]:
    """
    Exporte l'analyse Reddit au format TXT
    
    Args:
        analysis_text: Texte de l'analyse à exporter
        filename: Nom du fichier (défaut: reddit_analysis.txt)
    
    Returns:
        Dict avec le statut et le chemin du fichier
    """
    try:
        # Créer le dossier exports s'il n'existe pas
        export_dir = Path("exports")
        export_dir.mkdir(exist_ok=True)
        
        filepath = export_dir / filename
        
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(f"ANALYSE REDDIT - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("=" * 50 + "\n\n")
            f.write(analysis_text)
        
        return {
            "success": True,
            "filepath": str(filepath),
            "message": f"✅ Analyse exportée en TXT: {filepath}"
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

@function_tool
def export_analysis_to_csv(analysis_data: Dict, filename: str = "reddit_analysis.csv") -> Dict[str, str]:
    """
    Exporte l'analyse Reddit au format CSV
    
    Args:
        analysis_data: Données structurées de l'analyse
        filename: Nom du fichier (défaut: reddit_analysis.csv)
    
    Returns:
        Dict avec le statut et le chemin du fichier
    """
    try:
        export_dir = Path("exports")
        export_dir.mkdir(exist_ok=True)
        
        filepath = export_dir / filename
        
        with open(filepath, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Section', 'Contenu'])
            
            # Structurer les données pour CSV
            sections = analysis_data.get('sections', {})
            for section, content in sections.items():
                writer.writerow([section, content])
        
        return {
            "success": True,
            "filepath": str(filepath),
            "message": f"✅ Analyse exportée en CSV: {filepath}"
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

@function_tool
def export_analysis_to_pdf(analysis_text: str, filename: str = "reddit_analysis.pdf") -> Dict[str, str]:
    """
    Exporte l'analyse Reddit au format PDF
    
    Args:
        analysis_text: Texte de l'analyse à exporter
        filename: Nom du fichier (défaut: reddit_analysis.pdf)
    
    Returns:
        Dict avec le statut et le chemin du fichier
    """
    try:
        export_dir = Path("exports")
        export_dir.mkdir(exist_ok=True)
        
        filepath = export_dir / filename
        
        doc = SimpleDocTemplate(str(filepath), pagesize=letter)
        styles = getSampleStyleSheet()
        story = []
        
        # Titre
        title = Paragraph(f"ANALYSE REDDIT - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", styles['Title'])
        story.append(title)
        story.append(Spacer(1, 12))
        
        # Contenu
        content = Paragraph(analysis_text.replace('\n', '<br/>'), styles['Normal'])
        story.append(content)
        
        doc.build(story)
        
        return {
            "success": True,
            "filepath": str(filepath),
            "message": f"✅ Analyse exportée en PDF: {filepath}"
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

In [ ]:
# Créer l'agent spécialisé pour l'analyse Reddit
reddit_analyzer_agent = Agent(
    name="RedditAnalyzer",
    instructions="""Tu es un expert analyste spécialisé dans l'analyse de contenu Reddit.

Ton rôle est de:
1. Scraper les posts d'un subreddit spécifié
2. Analyser le contenu des posts et commentaires
3. Fournir un résumé structuré et des insights

Instructions spécifiques:
- Demande toujours le nom du subreddit et le nombre de posts à analyser
- Utilise l'outil scrape_subreddit_top_posts pour récupérer les données
- Analyse les thèmes récurrents, problèmes mentionnés, sentiments
- Fournis un résumé en français avec:
  * Thèmes principaux identifiés
  * Problèmes récurrents
  * Insights intéressants
  * Recommandations pour les entrepreneurs

Sois toujours professionnel et fournis des analyses utiles.""",
    tools=[
        scrape_subreddit_top_posts,
        WebSearchTool(),
        check_subreddit_exists
    ]
)

# Fonction pour lancer l'analyse
async def analyze_subreddit(subreddit_name: str, num_posts: int = 10) -> str:
    """
    Analyse complète d'un subreddit avec résumé
    
    Args:
        subreddit_name: Nom du subreddit
        num_posts: Nombre de posts à analyser
    
    Returns:
        Résumé d'analyse structuré
    """
    try:
        # Construire le prompt d'analyse
        analysis_prompt = f"""
        Analyse le subreddit r/{subreddit_name} en récupérant les {num_posts} posts avec le meilleur score.
        
        Fournis un résumé structuré incluant:
        1. 📊 Vue d'ensemble du subreddit
        2. 🎯 Thèmes principaux identifiés
        3. ⚠️ Problèmes récurrents mentionnés
        4. 💡 Insights intéressants pour les entrepreneurs
        5. �� Opportunités identifiées
        6. 📈 Tendances observées
        
        Sois précis et utilise les données réelles scrapées.
        """
        
        with trace("reddit_analysis"):
            result = await Runner.run(reddit_analyzer_agent, analysis_prompt)
            return result.final_output
            
    except Exception as e:
        return f"❌ Erreur lors de l'analyse: {str(e)}"

# Test de l'analyse
print("🔍 Test de l'analyse Reddit:")
test_analysis = await analyze_subreddit("entrepreneur", 5)
print("=" * 50)
print("RÉSULTAT DE L'ANALYSE:")
print("=" * 50)
print(test_analysis)

In [ ]:
# Interface Gradio pour l'analyse Reddit - Version corrigée
async def reddit_analysis_interface(message, history):
    """
    Interface pour analyser un subreddit
    """
    try:
        # Extraire les paramètres du message
        # Format attendu: "subreddit: nom_du_subreddit, posts: nombre"
        if ":" in message:
            parts = message.split(",")
            subreddit_name = "entrepreneur"  # défaut
            num_posts = 10  # défaut
            
            for part in parts:
                if "subreddit:" in part.lower():
                    subreddit_name = part.split(":")[1].strip()
                elif "posts:" in part.lower():
                    try:
                        num_posts = int(part.split(":")[1].strip())
                    except:
                        num_posts = 10
        else:
            # Si pas de format spécifique, utiliser le message comme nom de subreddit
            subreddit_name = message.strip()
            num_posts = 10
        
        # Construire le prompt d'analyse
        analysis_prompt = f"""
        Analyse le subreddit r/{subreddit_name} en récupérant les {num_posts} posts avec le meilleur score.
        
        Fournis un résumé structuré incluant:
        1. 📊 Vue d'ensemble du subreddit
        2. 🎯 Thèmes principaux identifiés
        3. ⚠️ Problèmes récurrents mentionnés
        4. 💡 Insights intéressants pour les entrepreneurs
        5. �� Opportunités identifiées
        6. 📈 Tendances observées
        
        Sois précis et utilise les données réelles scrapées.
        """
        
        with trace("gradio_analysis"):
            result = await Runner.run(reddit_analyzer_agent, analysis_prompt)
            return result.final_output
            
    except Exception as e:
        return f"❌ Erreur: {str(e)}"

# Interface Gradio corrigée
with gr.Blocks(title="Analyseur Reddit") as demo:
    gr.Markdown("# 🔍 Analyseur de Subreddits")
    gr.Markdown("Analysez les posts populaires d'un subreddit pour identifier des opportunités entrepreneuriales")
    
    # Interface de chat avec format messages
    chatbot = gr.ChatInterface(
        reddit_analysis_interface,
        title="Analyse Reddit",
        description="Tapez le nom d'un subreddit ou utilisez le format: 'subreddit: nom, posts: nombre'",
        examples=[
            "entrepreneur",
            "startup",
            "smallbusiness",
            "subreddit: programming, posts: 15",
            "subreddit: freelance, posts: 8"
        ],
        type="messages"  # Format moderne
    )
    
    # Instructions d'utilisation
    gr.Markdown("### 📝 Instructions d'utilisation:")
    gr.Markdown("- **Simple** : Tapez juste le nom du subreddit (ex: 'entrepreneur')")
    gr.Markdown("- **Avancé** : Utilisez le format 'subreddit: nom, posts: nombre'")
    gr.Markdown("- **Exemples** : Cliquez sur les exemples ci-dessus")
    
    gr.Markdown("### 🎯 Subreddits populaires à analyser:")
    gr.Markdown("- **entrepreneur** - Problèmes d'entrepreneurs")
    gr.Markdown("- **startup** - Défis des startups")
    gr.Markdown("- **smallbusiness** - Petites entreprises")
    gr.Markdown("- **freelance** - Freelancing et indépendants")
    gr.Markdown("- **programming** - Problèmes techniques")

demo.launch()

In [ ]:
# Version alternative avec inputs séparés
async def analyze_with_inputs(subreddit_name: str, num_posts: int):
    """
    Analyse avec paramètres séparés
    """
    try:
        analysis_prompt = f"""
        Analyse le subreddit r/{subreddit_name} en récupérant les {num_posts} posts avec le meilleur score.
        
        Fournis un résumé structuré incluant:
        1. 📊 Vue d'ensemble du subreddit
        2. 🎯 Thèmes principaux identifiés
        3. ⚠️ Problèmes récurrents mentionnés
        4. 💡 Insights intéressants pour les entrepreneurs
        5. �� Opportunités identifiées
        6. 📈 Tendances observées
        """
        
        with trace("analysis_with_inputs"):
            result = await Runner.run(reddit_analyzer_agent, analysis_prompt)
            return result.final_output
            
    except Exception as e:
        return f"❌ Erreur: {str(e)}"

# Interface alternative
with gr.Blocks(title="Analyseur Reddit - Version Simple") as demo:
    gr.Markdown("# 🔍 Analyseur de Subreddits")
    
    with gr.Row():
        with gr.Column():
            subreddit_input = gr.Textbox(
                label="Nom du subreddit",
                placeholder="ex: entrepreneur",
                value="entrepreneur"
            )
            posts_slider = gr.Slider(
                minimum=1,
                maximum=20,
                value=10,
                step=1,
                label="Nombre de posts"
            )
            analyze_btn = gr.Button("�� Analyser", variant="primary")
        
        with gr.Column():
            output_text = gr.Textbox(
                label="Résultat de l'analyse",
                lines=20,
                interactive=False
            )
    
    # Connecter le bouton
    analyze_btn.click(
        analyze_with_inputs,
        inputs=[subreddit_input, posts_slider],
        outputs=output_text
    )

demo.launch()

In [ ]:
@function_tool
def analyze_user_pains_from_posts(posts_data: List[Dict]) -> Dict[str, Any]:
    """
    Outil pour analyser les douleurs utilisateurs à partir de posts Reddit
    
    Args:
        posts_data: Liste des posts scrapés par reddit_analyzer_agent
    
    Returns:
        Dict avec les douleurs identifiées, scores et résumé
    """
    try:
        pain_patterns = {
            "problème": ["problème", "difficile", "compliqué", "embêtant"],
            "frustration": ["frustré", "énervé", "agacé", "exaspéré"],
            "difficulté": ["galère", "casse-tête", "obstacle", "blocage"],
            "stress": ["stressé", "inquiet", "anxieux", "paniqué"],
            "confusion": ["perdu", "confus", "pas clair", "incompréhensible"],
            "coût": ["cher", "coûteux", "budget", "prix"],
            "temps": ["long", "lent", "chronophage", "perte de temps"],
            "technique": ["bug", "erreur", "plantage", "dysfonctionnement"]
        }
        
        pain_analysis = {}
        
        for post in posts_data:
            text_to_analyze = f"{post['title']} {post['selftext']}".lower()
            
            for pain_type, keywords in pain_patterns.items():
                count = sum(1 for keyword in keywords if keyword in text_to_analyze)
                if count > 0:
                    if pain_type not in pain_analysis:
                        pain_analysis[pain_type] = {
                            "posts": [],
                            "total_upvotes": 0,
                            "total_comments": 0,
                            "frequency": 0
                        }
                    
                    pain_analysis[pain_type]["posts"].append(post)
                    pain_analysis[pain_type]["total_upvotes"] += post["score"]
                    pain_analysis[pain_type]["total_comments"] += post["num_comments"]
                    pain_analysis[pain_type]["frequency"] += 1
        
        # Calculer les scores
        pain_scores = []
        for pain_type, data in pain_analysis.items():
            if data["frequency"] > 0:
                avg_upvotes = data["total_upvotes"] / data["frequency"]
                avg_comments = data["total_comments"] / data["frequency"]
                score = (data["frequency"] * 0.6) + (avg_upvotes * 0.3) + (avg_comments * 0.1)
                
                pain_scores.append({
                    "pain_type": pain_type,
                    "frequency": data["frequency"],
                    "avg_upvotes": round(avg_upvotes, 2),
                    "avg_comments": round(avg_comments, 2),
                    "score": round(score, 2),
                    "posts_count": len(data["posts"])
                })
        
        pain_scores.sort(key=lambda x: x["score"], reverse=True)
        
        return {
            "success": True,
            "total_posts_analyzed": len(posts_data),
            "pains_identified": len(pain_scores),
            "top_pains": pain_scores[:5],
            "all_pains": pain_scores
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

# Agent mis à jour avec l'outil d'analyse des douleurs
reddit_analyzer_agent = Agent(
    name="RedditAnalyzer",
    instructions="""Tu es un expert analyste spécialisé dans l'analyse de contenu Reddit.

Ton rôle est de:
1. Scraper les posts d'un subreddit spécifié
2. Analyser le contenu des posts et commentaires
3. Identifier les douleurs utilisateurs récurrentes
4. Fournir un résumé structuré et des insights

Instructions spécifiques:
- Demande toujours le nom du subreddit et le nombre de posts à analyser
- Utilise scrape_subreddit_posts pour récupérer les données
- Utilise analyze_user_pains_from_posts pour identifier les douleurs
- Fournis un résumé en français avec:
  * Vue d'ensemble du subreddit
  * Top 5 douleurs utilisateurs identifiées
  * Scores de priorité pour chaque douleur
  * Insights intéressants pour les entrepreneurs
  * Opportunités identifiées
- À la fin, propose d'exporter en TXT, CSV ou PDF

Méthodologie de scoring des douleurs:
- Fréquence = nombre de posts mentionnant cette douleur
- Intérêt = moyenne des upvotes et commentaires
- Score final = (fréquence * 0.6) + (moyenne_upvotes * 0.3) + (moyenne_commentaires * 0.1)

Sois toujours professionnel et fournis des analyses utiles.""",
    tools=[
        scrape_subreddit_posts,
        analyze_user_pains_from_posts,
        WebSearchTool(),
        check_subreddit_exists,
        export_analysis_to_txt,
        export_analysis_to_csv,
        export_analysis_to_pdf
    ]
)

In [ ]:
# Agent spécialisé dans l'identification des douleurs utilisateurs
pain_identifier_agent = Agent(
    name="PainIdentifier",
    instructions="""Tu es un expert analyste spécialisé dans l'identification des problèmes et frustrations récurrents des utilisateurs.

Ton rôle est de:
1. Analyser le sentiment et l'intensité émotionnelle de chaque post/commentaire
2. Identifier les douleurs récurrentes indépendamment de la langue
3. Lister les 5 principales douleurs en une phrase chacune
4. Calculer un score de priorité pour chaque douleur
5. Classer les douleurs par ordre de priorité

Instructions spécifiques:
- Analyse le SENTIMENT (frustration, colère, stress, confusion, etc.)
- Évalue l'INTENSITÉ émotionnelle (1-10, 10 = très intense)
- Identifie le TYPE de problème (technique, financier, relationnel, etc.)
- Regroupe les douleurs similaires
- Calcule les métriques pour chaque douleur
- Présente les top 3 douleurs avec leurs scores

Format de sortie attendu:
1. Liste des 5 principales douleurs (une phrase chacune)
2. Top 3 classé par score
3. Résumé en un paragraphe
4. Recommandations pour les entrepreneurs

Sois précis dans l'identification émotionnelle et le scoring des problèmes.""",
    tools=[]
)

@function_tool
def identify_user_pains_llm(posts_data: List[Dict]) -> Dict[str, Any]:
    """
    Outil pour identifier les douleurs utilisateurs via analyse LLM
    
    Args:
        posts_data: Liste des posts scrapés
    
    Returns:
        Dict avec les douleurs identifiées, scores et résumé
    """
    try:
        # Préparer les données pour l'analyse LLM
        posts_for_analysis = []
        
        for post in posts_data:
            # Combiner titre, contenu et commentaires
            full_text = f"Titre: {post['title']}\n"
            if post.get('selftext'):
                full_text += f"Contenu: {post['selftext']}\n"
            
            # Ajouter les commentaires principaux
            if post.get('comments'):
                full_text += "Commentaires:\n"
                for comment in post['comments'][:5]:  # Top 5 commentaires
                    full_text += f"- {comment.get('body', '')}\n"
            
            posts_for_analysis.append({
                "text": full_text,
                "score": post["score"],
                "num_comments": post["num_comments"],
                "url": post["url"]
            })
        
        # Analyser chaque post avec le LLM
        pain_analysis = {}
        
        for i, post in enumerate(posts_for_analysis):
            analysis_prompt = f"""
            Analyse ce post Reddit pour identifier les douleurs utilisateurs.
            
            POST #{i+1}:
            {post['text']}
            
            Analyse ce post et réponds au format JSON:
            {{
                "sentiment": "frustration/colère/stress/confusion/autre",
                "intensity": 1-10,
                "pain_types": ["type1", "type2"],
                "description": "description courte de la douleur"
            }}
            
            Règles:
            - sentiment: type d'émotion principale
            - intensity: 1-10 (10 = très intense)
            - pain_types: types de problèmes (technique, financier, relationnel, temps, etc.)
            - description: résumé en une phrase
            """
            
            with trace(f"post_analysis_{i}"):
                result = await Runner.run(pain_analyzer_agent, analysis_prompt)
                
                try:
                    # Parser la réponse JSON
                    analysis = json.loads(result.final_output)
                    
                    # Traiter chaque type de douleur identifié
                    for pain_type in analysis.get('pain_types', []):
                        if pain_type not in pain_analysis:
                            pain_analysis[pain_type] = {
                                "posts": [],
                                "total_upvotes": 0,
                                "total_comments": 0,
                                "frequency": 0,
                                "total_intensity": 0,
                                "descriptions": []
                            }
                        
                        pain_analysis[pain_type]["posts"].append(post)
                        pain_analysis[pain_type]["total_upvotes"] += post["score"]
                        pain_analysis[pain_type]["total_comments"] += post["num_comments"]
                        pain_analysis[pain_type]["frequency"] += 1
                        pain_analysis[pain_type]["total_intensity"] += analysis.get('intensity', 5)
                        pain_analysis[pain_type]["descriptions"].append(analysis.get('description', ''))
                
                except json.JSONDecodeError:
                    # Si le LLM ne retourne pas du JSON valide, continuer
                    continue
        
        # Calculer les scores pour chaque douleur
        pain_scores = []
        for pain_type, data in pain_analysis.items():
            if data["frequency"] > 0:
                avg_upvotes = data["total_upvotes"] / data["frequency"]
                avg_comments = data["total_comments"] / data["frequency"]
                avg_intensity = data["total_intensity"] / data["frequency"]
                
                # Score pondéré incluant l'intensité émotionnelle
                score = (data["frequency"] * 0.4) + (avg_upvotes * 0.2) + (avg_comments * 0.1) + (avg_intensity * 0.3)
                
                pain_scores.append({
                    "pain_type": pain_type,
                    "frequency": data["frequency"],
                    "avg_upvotes": round(avg_upvotes, 2),
                    "avg_comments": round(avg_comments, 2),
                    "avg_intensity": round(avg_intensity, 2),
                    "score": round(score, 2),
                    "posts_count": len(data["posts"]),
                    "descriptions": data["descriptions"][:3]  # Top 3 descriptions
                })
        
        pain_scores.sort(key=lambda x: x["score"], reverse=True)
        
        # Résumé final avec le LLM
        summary_prompt = f"""
        Analyse les douleurs utilisateurs identifiées dans les posts Reddit.
        
        Données d'analyse:
        - Posts analysés: {len(posts_data)}
        - Douleurs identifiées: {len(pain_scores)}
        
        Top 5 douleurs par score:
        {json.dumps(pain_scores[:5], indent=2, ensure_ascii=False)}
        
        Tâches:
        1. Liste les 5 principales douleurs en une phrase chacune
        2. Présente les top 3 avec leurs scores détaillés
        3. Résume les problèmes en un court paragraphe
        4. Donne des recommandations pour les entrepreneurs
        
        Format de réponse:
        ## Top 5 Douleurs Identifiées
        1. [Douleur 1] - Score: [X] (Intensité: [Y])
        2. [Douleur 2] - Score: [X] (Intensité: [Y])
        ...
        
        ## Top 3 Priorités
        1. [Douleur principale] - Score: [X] (Fréquence: [Y], Intensité: [Z])
        2. [Deuxième douleur] - Score: [X] (Fréquence: [Y], Intensité: [Z])
        3. [Troisième douleur] - Score: [X] (Fréquence: [Y], Intensité: [Z])
        
        ## Résumé
        [Paragraphe de résumé]
        
        ## Recommandations
        [Recommandations pour les entrepreneurs]
        """
        
        with trace("final_analysis"):
            final_result = await Runner.run(pain_identifier_agent, summary_prompt)
            
            return {
                "success": True,
                "raw_data": {
                    "total_posts_analyzed": len(posts_data),
                    "pains_identified": len(pain_scores),
                    "top_pains": pain_scores[:5],
                    "all_pains": pain_scores
                },
                "analysis_summary": final_result.final_output,
                "top_3_pains": pain_scores[:3]
            }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }